In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

## Reading the spreadsheet

In [3]:
excel_path = "BaseDadosNotas.xlsx"
df = pd.read_excel(excel_path)
df.head()

,ID,PROVA1,PROVA2,TRABALHO,SITUAÇÃO
0,a1902_01,2.0,1.5,0.5,Reprovado
1,a1902_02,8.0,4.8,8.8,Aprovado
2,a1902_03,3.0,0.0,1.8,Reprovado
3,a1902_04,8.0,7.5,2.3,Aprovado
4,a1902_05,2.5,0.0,1.2,Reprovado


## Feature Engineering

In [6]:
# Separar as features
X = df[['PROVA1', 'PROVA2', 'TRABALHO']]  # define as entradas
y = df['SITUAÇÃO'].map({'Aprovado': 1, 'Reprovado': 0})  # define as saidas (boolean)

# divide os dados (148 para treino, 73 para teste)
X_train, X_test = X.iloc[:148], X.iloc[148:]
y_train, y_test = y.iloc[:148], y.iloc[148:]

# Converte os dados para tensores do tipo float para as features
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Converte os targets para tensores (pode ser float para a função BCELoss)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [8]:
class Perceptron(nn.Module):
    def __init__(self, input_size, peso, bias, use_bias):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, 1, bias=use_bias)      # Camada linear: transforma entrada em saída

        '''
        # Definir pesos e viés iniciais
        with torch.no_grad():
            self.fc.weight.fill_(peso)      # Define o peso
            if use_bias:
                self.fc.bias.fill_(bias)        # Define o viés
        #OBS: o peso e o bias são definidos aleatóriamente ao iniciar nn.Linear
        '''

    #A função forward define o que acontece quando os dados passam pelo modelo, ou seja, como a entrada é transformada em saída.
    def forward(self, x):
        return torch.sigmoid(self.fc(x))                    # Usa a função sigmoide para limitar a saída entre 0 e 1

In [9]:
# Rede 3 neuronios
class RedeNeural:
    def __init__(self):
        input_size = 3
        peso = 0.5
        bias = 0.5
        use_bias = True

        self.perceptron_entrada1 = Perceptron(input_size, peso, bias, use_bias)
        self.perceptron_entrada2 = Perceptron(input_size, peso, bias, use_bias)
        self.perceptron_entrada3 = Perceptron(input_size, peso, bias, use_bias)
        self.perceptron_saida1 = Perceptron(3, peso, bias, use_bias)

        lr = 0.5
        self.criterion = nn.BCELoss()
        self.optimizer_entrada1 = optim.SGD(self.perceptron_entrada1.parameters(), lr)
        self.optimizer_entrada2 = optim.SGD(self.perceptron_entrada2.parameters(), lr)
        self.optimizer_entrada3 = optim.SGD(self.perceptron_entrada3.parameters(), lr)
        self.optimizer_saida1 = optim.SGD(self.perceptron_saida1.parameters(), lr)

    def treinamento(self, epocas):
        for epoch in range(epocas):
            self.optimizer_entrada1.zero_grad()
            self.optimizer_entrada2.zero_grad()
            self.optimizer_entrada3.zero_grad()
            self.optimizer_saida1.zero_grad()

            output_1 = self.perceptron_entrada1(X_train_tensor)
            output_2 = self.perceptron_entrada2(X_train_tensor)
            output_3 = self.perceptron_entrada3(X_train_tensor)

            combined = torch.cat((output_1, output_2, output_3), dim=1)
            output_final = self.perceptron_saida1(combined)
            loss_final = self.criterion(output_final, y_train_tensor)
            loss_total = loss_final
            loss_total.backward()

            self.optimizer_entrada1.step()
            self.optimizer_entrada2.step()
            self.optimizer_entrada3.step()
            self.optimizer_saida1.step()

    def teste(self):
        with torch.no_grad():
            output_1 = self.perceptron_entrada1(X_test_tensor)
            output_2 = self.perceptron_entrada2(X_test_tensor)
            output_3 = self.perceptron_entrada3(X_test_tensor)
            output_final = self.perceptron_saida1(torch.cat((output_1, output_2, output_3), dim=1))
        saida_binaria = (output_final >= 0.5).float()
        return saida_binaria, output_final

In [10]:
class RedeNeuralMLP2:

    def __init__(self):
        # Inicializar o Perceptron
        input_size = 3                         #número de neurônios de entrada (prova1, 2 e trabalho)

        #define os valores iniciais das variáveis
        peso = 0.5
        bias = 0.5
        use_bias = True


        # cria os perceptrons de entrada (camada oculta)
        self.perceptron_entrada1 = Perceptron(input_size, peso, bias, use_bias)
        self.perceptron_entrada2 = Perceptron(input_size, peso, bias, use_bias)
        #self.perceptron_entrada3 = Perceptron(input_size, peso, bias, use_bias)
        # perceptron de saida
        self.perceptron_saida1 = Perceptron(2, peso, bias, use_bias)    #input_size deve ser 2 pois ele receberá as saídas de ambos os neurônios anteriores

        #taxa de aprendizagem
        lr = 0.5

        # Função de perda e otimizador
        self.criterion = nn.BCELoss()                        #Serve para calcular a função de erro(loss) para o backpropagation

        #servem para atualizar os pesos (otimizar)
        self.optimizer_entrada1 = optim.SGD(self.perceptron_entrada1.parameters(), lr)   # Gradiente descendente estocástico
        self.optimizer_entrada2 = optim.SGD(self.perceptron_entrada2.parameters(), lr)   # Gradiente descendente estocástico
        #self.optimizer_entrada3 = optim.SGD(self.perceptron_entrada3.parameters(), lr)  # Gradiente descendente estocástico
        self.optimizer_saida1 = optim.SGD(self.perceptron_saida1.parameters(), lr)   # Gradiente descendente estocástico

    def treinamento(self, epocas):
        for epoch in range(epocas):
            # Zera os gradientes para todas as partes da rede
            self.optimizer_entrada1.zero_grad()
            self.optimizer_entrada2.zero_grad()
            self.optimizer_saida1.zero_grad()

            # Passagem forward dos perceptrons de entrada
            output_1 = self.perceptron_entrada1(X_train_tensor)
            output_2 = self.perceptron_entrada2(X_train_tensor)

            # Calcula as perdas individuais para as camadas de entrada
            #loss1 = self.criterion(output_1, y_train_tensor)
            #loss2 = self.criterion(output_2, y_train_tensor)

            # Faz com que o neurônio de saída receba ambas as entradas separadamente, a fim de poder pondera-las corretamente
            combined = torch.cat((output_1, output_2), dim=1)
            output_final = self.perceptron_saida1(combined)
            #output_final = self.perceptron_saida1((output_1 + output_2) / 2)
            loss_final = self.criterion(output_final, y_train_tensor)

            # Soma todas as perdas para formar a perda total
            loss_total = loss_final

            # Realiza a retropropagação de uma única vez
            loss_total.backward()

            # Atualiza os pesos
            self.optimizer_entrada1.step()
            self.optimizer_entrada2.step()
            self.optimizer_saida1.step()

            # Opcional: imprimir os losses para monitorar o treinamento
            if (epoch + 1) % 10 == 0:
                print(
                    f"Época {epoch + 1}/{epocas}, Loss final: {loss_final.item():.4f}"
                )

    def teste(self):
        # Desativa o cálculo de gradientes para avaliação
        with torch.no_grad():
            output_1 = self.perceptron_entrada1(X_test_tensor)
            output_2 = self.perceptron_entrada2(X_test_tensor)
            output_final = self.perceptron_saida1(torch.cat((output_1, output_2), dim=1))

        # Aplica threshold 0.5 para converter probabilidades em classe binária
        saida_binaria = (output_final >= 0.5).float()
        return saida_binaria, output_final

In [11]:
def predict(rn):
    rn.treinamento(epocas)
    y_pred_bin, y_pred_prob = rn.teste()
    pred_bin_np = y_pred_bin.squeeze().numpy()
    pred_prob_np = y_pred_prob.squeeze().numpy()
    y_true_np = y_test_tensor.squeeze().numpy()
    acertou = (pred_bin_np == y_true_np)
    acertos = acertou.sum()
    total = len(acertou)
    taxa_acerto = acertos / total
    return [epocas, i + 1, acertos, total, taxa_acerto]

In [12]:
resultados = []

for epocas in [30, 50, 100]:
    for i in range(50):
        rn_3neuronios = RedeNeural()
        resultado = predict(rn_3neuronios)
        resultados.append(resultado)


df_resultados = pd.DataFrame(resultados, columns=["Épocas", "Execução", "Acertos", "Total", "Taxa de Acerto"])
df_resultados.to_csv("resultados.csv", sep="\t", index=False)
df_resultados.to_excel("resultados.xlsx", index=False)
estatisticas = df_resultados.groupby("Épocas")["Taxa de Acerto"].describe()
estatisticas.to_excel("analise_estatistica.xlsx")
estatisticas

,count,mean,std,min,25%,50%,75%,max
Épocas,,,,,,,,
30,50.0,0.670411,0.017593,0.657534,0.657534,0.657534,0.671233,0.726027
50,50.0,0.704384,0.036725,0.657534,0.671233,0.712329,0.739726,0.753425
100,50.0,0.741918,0.005073,0.739726,0.739726,0.739726,0.739726,0.753425


In [13]:
resultados2 = []

for epocas in [30, 50, 100]:
    for i in range(50):
        rn_MLP2 = RedeNeuralMLP2()
        resultado = predict(rn_MLP2)
        resultados2.append(resultado)


df_resultados2 = pd.DataFrame(resultados2, columns=["Épocas", "Execução", "Acertos", "Total", "Taxa de Acerto"])
df_resultados2.to_csv("resultados2.csv", sep="\t", index=False)
df_resultados2.to_excel("resultados2.xlsx", index=False)
estatisticas2 = df_resultados2.groupby("Épocas")["Taxa de Acerto"].describe()
estatisticas2.to_excel("analise_estatistica2.xlsx")
estatisticas2

Época 10/30, Loss final: 0.5637
Época 20/30, Loss final: 0.5375
Época 30/30, Loss final: 0.4984
Época 10/30, Loss final: 0.5515
Época 20/30, Loss final: 0.5302
Época 30/30, Loss final: 0.5028
Época 10/30, Loss final: 0.5905
Época 20/30, Loss final: 0.5857
Época 30/30, Loss final: 0.5740
Época 10/30, Loss final: 0.5988
Época 20/30, Loss final: 0.5905
Época 30/30, Loss final: 0.5853
Época 10/30, Loss final: 0.5970
Época 20/30, Loss final: 0.5810
Época 30/30, Loss final: 0.5650
Época 10/30, Loss final: 0.6053
Época 20/30, Loss final: 0.5613
Época 30/30, Loss final: 0.5318
Época 10/30, Loss final: 0.5815
Época 20/30, Loss final: 0.5259
Época 30/30, Loss final: 0.4904
Época 10/30, Loss final: 0.5744
Época 20/30, Loss final: 0.5580
Época 30/30, Loss final: 0.5417
Época 10/30, Loss final: 0.5973
Época 20/30, Loss final: 0.5800
Época 30/30, Loss final: 0.5656
Época 10/30, Loss final: 0.5794
Época 20/30, Loss final: 0.5532
Época 30/30, Loss final: 0.5306
Época 10/30, Loss final: 0.5624
Época 20

,count,mean,std,min,25%,50%,75%,max
Épocas,,,,,,,,
30,50.0,0.665205,0.018182,0.657534,0.657534,0.657534,0.657534,0.726027
50,50.0,0.699726,0.036907,0.657534,0.657534,0.698630,0.736301,0.753425
100,50.0,0.739178,0.012969,0.657534,0.739726,0.739726,0.739726,0.767123
